# JupyterLab Widgets Test
Note: [vim-keybindings](https://github.com/jwkvam/jupyterlab_vim) are enabled by default in this repo.

In [ ]:
import numpy as np
import ipywidgets as ipw
import bqplot as bq
import ipyvolume as ipv

# IPyWidgets

In [ ]:
ipw.interact(lambda x: x, x=True);

In [ ]:
ipw.IntSlider()

# bqplot

In [ ]:
x_sc = bq.LinearScale()
y_sc = bq.LinearScale()

x_data = np.arange(20)
y_data = np.random.randn(20)

scatter_chart = bq.Scatter(x=x_data, y=y_data, scales= {'x': x_sc, 'y': y_sc}, colors=['dodgerblue'],
                       interactions={'click': 'select'},
                        selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                       unselected_style={'opacity': 0.5})

ax_x = bq.Axis(scale=x_sc)
ax_y = bq.Axis(scale=y_sc, orientation='vertical', tick_format='0.2f')

bq.Figure(marks=[scatter_chart], axes=[ax_x, ax_y])

# IPyVolume

In [ ]:
t = np.linspace(0, 1, 21)
x, y, z = np.meshgrid(t,t,t, indexing='ij')
f = x**2 - y + np.sin(2*np.pi*z)

ipv.figure()
ipv.volshow(f)
ipv.plot_isosurface(f.T, color='gray')
ipv.show()

# Plotly

In [ ]:
# From https://plot.ly/python/3d-network-graph/
import igraph as ig
import json
import urllib.request as urllib2

data = []
req = urllib2.Request("https://raw.githubusercontent.com/plotly/datasets/master/miserables.json")
opener = urllib2.build_opener()
f = opener.open(req)
data = json.loads(f.read().decode())

N=len(data['nodes'])

L=len(data['links'])
Edges=[(data['links'][k]['source'], data['links'][k]['target']) for k in range(L)]

G=ig.Graph(Edges, directed=False)

labels=[]
group=[]
for node in data['nodes']:
    labels.append(node['name'])
    group.append(node['group'])

layt=G.layout('kk', dim=3)

Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]



import plotly.offline as py
py.init_notebook_mode()
from plotly.graph_objs import *

trace1=Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=Line(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )
trace2=Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=Marker(symbol='dot',
                             size=6,
                             color=group,
                             colorscale='Viridis',
                             line=Line(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = Layout(
         title="Network of coappearances of characters in Victor Hugo's novel<br> Les Miserables (3D visualization)",
         width=800,
         height=600,
         showlegend=False,
         scene=Scene(
         xaxis=XAxis(axis),
         yaxis=YAxis(axis),
         zaxis=ZAxis(axis),
        ),
     margin=Margin(
        t=100
    ),
    hovermode='closest',
    annotations=Annotations([
           Annotation(
           showarrow=False,
            text="Data source: <a href='http://bost.ocks.org/mike/miserables/miserables.json'>[1] miserables.json</a>",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=Font(
            size=14
            )
            )
        ]),    )

data=Data([trace1, trace2])
fig=Figure(data=data, layout=layout)

py.iplot(fig, filename='Les-Miserables')

# IPyLeaflet

In [ ]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

from IPython.display import display
from traitlets import link

center = [34.6252978589571, -77.34580993652344]
zoom = 10

m = Map(center=center, zoom=zoom)

dc = DrawControl()

def handle_draw(self, action, geo_json):
    print(action)
    print(geo_json)

dc.on_draw(handle_draw)
m.add_control(dc)

m2 = Map(center=center, zoom=zoom, layout=dict(width='600px', height='400px'))

map_center_link = link((m, 'center'), (m2, 'center'))
map_zoom_link = link((m, 'zoom'), (m2, 'zoom'))

new_poly = GeoJSON(data=dc.last_draw)

m2.add_layer(new_poly)
display(ipw.VBox([m, m2]))

# Woohoo!

So many widgets!